In [42]:
from pymol import cmd, stored
import pandas as pd, numpy as np

In [ ]:
# load the pdbs and fix the objects, must use reinitilize if session already has loaded object
# assumes files are in same directory as the notebook, update paths otherwise
cmd.reinitialize()
pdbs = {"1uwf":"1uwf.cif","3zpd":"3zpd.cif","4xob":"4xob.cif"}
for pdb, filename in pdbs.items():
    cmd.load(filename,pdb)

cmd.create("4xob_FimH", "(4xob and segid A and i. 1-160) (4xob and segid I)")
cmd.delete("4xob")
cmd.set_name("4xob_FimH","4xob")

In [ ]:
cutoff = 3.5
angle = 45
hb = {}
for pdb, filename in pdbs.items():
    #cmd.load(filename,pdb)
    selection = pdb + " and n. n & e. n+o"
    selection2 = pdb + " and !n. n and !resn HOH & e. n+o"
    hb[pdb] = cmd.find_pairs(selection,selection2,mode=1,cutoff=cutoff,angle=angle)

cols = ['resn_1','resi_1','name_1','resn_2','resi_2','name_2','dist']
hb_df = pd.DataFrame()
for pdb in pdbs.keys():
    stored.pair1 = []
    stored.pair2 = []
    dists = []
    for pairs in hb[pdb]:
        cmd.iterate("%s and index %s" % (pairs[0][0],pairs[0][1]), 'stored.pair1.append((resn,resi,name))')
        cmd.iterate("%s and index %s" % (pairs[1][0],pairs[1][1]), 'stored.pair2.append((resn,resi,name))')
        dists.append(cmd.distance("hb_list","%s and index %s" % (pairs[0][0],pairs[0][1]),"%s and index %s" % (pairs[1][0],pairs[1][1])))
    x1 = stored.pair1
    x2 = stored.pair2
    x12 = [list(a1+a2)+[d] for a1,a2,d in zip(x1,x2,dists)]

    for idx,dist in enumerate(x12):       
        dist_dict = {cols[i]:[item] for i,item in enumerate(dist)}
        dist_dict['pair_idx'] = idx
        dist_dict['pdb'] = pdb
        hb_df = pd.concat([hb_df,pd.DataFrame.from_dict(dist_dict)],ignore_index=True)


hb_df = hb_df.apply(pd.to_numeric,errors='ignore')
hb_df = hb_df.sort_values(by=['pdb','resi_1'])
hb_df = hb_df.reset_index(drop=True)
hb_df

,resn_1,resi_1,name_1,resn_2,resi_2,name_2,dist,pair_idx,pdb
0,PHE,1,N,DEG,1159,O5,2.883179,104,1uwf
1,ALA,2,N,HIS,45,O,3.332876,25,1uwf
2,CYS,3,N,ILE,11,O,3.107933,7,1uwf
3,LYS,4,N,PHE,43,O,2.782293,23,1uwf
4,THR,5,N,THR,9,O,2.691800,6,1uwf
...,...,...,...,...,...,...,...,...,...
322,VAL,154,N,ILE,120,O,3.238791,76,4xob
323,VAL,156,N,VAL,118,O,2.883975,75,4xob
324,THR,158,N,GLY,116,O,2.713861,74,4xob
325,GLY,159,N,ASN,33,ND2,3.493001,18,4xob
